# Voting Mechanism Experiment Template

## Experiment #1 

(Use a unique identifier for the number next to "Experiment".)

## Experiment Overview

### Key Question

In Single-Choice Weighted Plurality, does adding weight for each NFTs produce different results (final winner) than one wallet, one vote?

### Procedure for Generating Data

A **trial** consists of the following steps:

**Step 1:** We load the actual TE Academy data available from the Otterspace subgraph, accessed on May 28. This gives us the real data available to voters. 

**Step 2:** We create two different weightings for the voters.
A. *One Wallet, One Vote:* each voter is assigned a weight of 1.0, regardless of how many NFTs they hold. (This is also *uniform weighting*.)
B. *Each NFT has a weight of 1.0*: each voter is assigned a weight equal to the number of NFTs they hold. Each NFT is worth 1.0.

**Step 3:** We randomly assign voter choices.

**Step 4:** We calculate the results (including final winner) for each weighting, and keep track of results in a DataFrame. 

We aim to calculate 100_000 trials. 

### Measurement

We measure which proportion of the time the two weightings produced the same winner. 

## Code for Simulation

In [10]:


# Standard Imports
import numpy as np
import pandas as pd
import os
from random import choice
from typing import Dict

import sys
sys.path.append('..')  # Add this line to include the directory above

# Custom imports
from custom_types import UserNFTs
from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality
from mechanisms.group_hug_mechanism import GroupHug

## Importing the Data 

In the cell below, we are importing the available data as of May 28, 2024 regarding which users have which NFTs. 

In [11]:
file_name = "../data/nft_data_may_28_2024_cleaned.csv"
file_exists = os.path.exists(file_name)
file_exists

True

Input a default voting weights csv (these were suggested by )

In [32]:
DEFAULT_WEIGHTS = pd.read_csv("../data/default_voting_weights.csv")
DEFAULT_WEIGHTS.columns = DEFAULT_WEIGHTS.columns.str.strip()

In [33]:
DEFAULT_WEIGHTS.columns

Index(['Name', 'Category', 'Count', 'Weight'], dtype='object')

In [34]:
DEFAULT_WEIGHTS.head(5)

,Name,Category,Count,Weight
0,TE FUNDAMENTALS COURSE AUTHOR LAUNCH 2022,POE,4.0,20.0
1,Token Engineering @EthCC Paris 2023 - Speaker,POE,7.0,16.0
2,TE FUNDAMENTALS 1,POK,332.0,7.0
3,TE FUNDAMENTALS 2,POK,249.0,7.0
4,TE FUNDAMENTALS 3,POK,168.0,7.0


In [35]:
DEFAULT_WEIGHTS.at[3, 'Count']

249.0

In [55]:
    # This is intended to replicate the spreadsheet formula =IF(COUNT=0,0,RANDBETWEEN(0,1))

    import random

    nft_credentials = DEFAULT_WEIGHTS['Name']

    def get_nft_count(nft_data: pd.DataFrame, 
                        row_number: int,
                        col_name: str): 
        return nft_data.at[row_number, col_name]
        
    def generate_nested_dict(nft_data: pd.DataFrame = DEFAULT_WEIGHTS, 
                             col_name: str = "Count",
                             voters=None,
                             credentials=None):
        nested_dictionary = {}
        for voter in voters:
            nested_dictionary[voter] = {}
            for row_number, credential in enumerate(credentials):
                actual_credential_count = get_nft_count(nft_data, 
                                                        row_number,
                                                        col_name)
                issued_credential_count = 0  # Start with 0
                if actual_credential_count > 0:  # Check if the credential has been issued
                    voter_is_issued_credential = random.choice([0, 1])
                    nested_dictionary[voter][credential] = voter_is_issued_credential
        return nested_dictionary

In [39]:
get_nft_count(nft_data = DEFAULT_WEIGHTS, 
              row_number = 3,
              col_name = 'Count')

249.0

In [57]:
x = 3
my_name = f"voter_{x}"

print(f"My name is {my_name}")

My name is voter_3


In [61]:
my_voter_names = [f"voter_{(k+1)}" for k in range(351)]

In [63]:
my_voters = enerate_nested_dict(voters = my_voter_names,
                     credentials = DEFAULT_WEIGHTS['Name'])

NameError: name 'enerate_nested_dict' is not defined

In [45]:
my_list = ["A","B","C","D"]

for k in my_list:
    print(k)

A
B
C
D


In [46]:
for idx, k in enumerate(my_list):
    print(f"The value of {idx} is {k}.")

The value of 0 is A.
The value of 1 is B.
The value of 2 is C.
The value of 3 is D.


# Key Question

How often would a vote that incorporates credibility-weighting by NFTs produce a different outcome than simply doing one wallet, one vote? We use the actual TE Academy NFT data as of May 28 to look at this question. 

## Inputting Sample Data 

In the cells below, we load the data available from the `.csv` file in the `VOTER_DATA_FILENAME`. 

If you wish to use a different `.csv` file, change this line. 

In [ ]:
VOTER_DATA_FILENAME = "../data/nft_data_may_28_2024_cleaned.csv" # Adjust data here. 

voter_data = pd.read_csv(VOTER_DATA_FILENAME) #Convert to a Pandas DataFrame. 

# Sometimes there is a 
if ['Unnamed: 0'] in voter_data.columns:
    voter_data.drop(columns = ['Unnamed: 0'], 
                    inplace = True)
voter_data.set_index('ID', inplace = True)

In [ ]:
sample_voters = voter_data.to_dict(orient='index')
sample_voters

In [ ]:
# NOTE: This next step isn't strictly necessary at this stage. 
voters = {key: UserNFTs(sample_voters.get(key))
          for key, _ in sample_voters.items()
          }

In [ ]:
voters

When working with data, it's often helpful to see a particular entry to see what it looks like. 

In [ ]:
first_voter = list(sample_voters.values())[0]

### Assigning Weights

We need to create a dictionary of weights for the NFTS. 
The dictionary `NFT_weights` gives a weight of 1.0 to each NFT held by current TEA owners.  

In [ ]:
NFT_weights = {key: 1.0 
               for key
               in first_voter.keys()
               }
NFT_weights

Single Choice Weighted Plurality requires that voters have some kind of weight. 
The function below takes a dictionary of voters with NFTs held, and a dictionary of weights.
It returns the weight that each voter should hold, based on 

In [ ]:
def calc_voter_weights_from_NFT_weight(voters: Dict,
                                       nft_weights: Dict) -> Dict:
    """
    Given a list of voters, the NFTs they hold, and the 
    """
    new_dict = {}
    for voter in voters.keys():
        new_dict[voter] = {}
        new_dict[voter]["weight"] = 0
        for nft_name, nft_val in nft_weights.items():
            if voters.get(voter).get(nft_name):
                new_dict[voter]["weight"] += nft_val

    return new_dict

Now we use the function to assign each voter a weight, based on the NFTs. 

In [ ]:
voter_weights = calc_voter_weights_from_NFT_weight(voters,
                                                   NFT_weights)
voter_weights

As a comparison, we also create a version where each voter is assigned the same weight of 1.0, regardless of how many NFTs they hold. 

In [ ]:
uniform_weights = {voter: {}"weight": 1.0} for voter in sample_voters.keys()}
uniform_weights

## Data Generation

The data generation is below. Each voter is randomly assigned a choice to vote for. 
The results of using SingleChoiceWeightedPlurality are calculated for each trial, based on these voter choices and weights. 
These results are stored in a DataFrame. 

In [ ]:
# Create the mechanism
SCWPCalculator = SingleChoiceWeightedPlurality()

# Set the number of experiments
NUM_EXPERIMENTS = 10_000

# Create an empty DataFrame to store simulation results
results_list = [] 

for k in range(NUM_EXPERIMENTS):
    # Generate choices for each voter
    sample_choices = {
                   key: choice(["A","B","C","D"])
                   for key in sample_voters.keys()
                 }
    
    # Calculate who would win with weight of 1 per each NFT
    weighted_winner, weighted_results  = SCWPCalculator.calculate(voter_weights,
                            sample_choices)
    # Calculate who would win with uniform weight 
    uniform_winner, uniform_results = SCWPCalculator.calculate(uniform_weights,
                            sample_choices)
                   
    # Create an empty DataFrame to store simulation results
    results_list.append({'Experiment': k,
                        'weighted_winner': weighted_winner, 
                        'weighted_candidate_scores': weighted_results,
                        'uniform_winner': uniform_winner,
                        'uniform_candidate_scores': uniform_results,
                        "votes": sample_choices
                        }
                        )


results_df = pd.DataFrame(results_list)

#TODO: Refactor for speed if needed.  

Let's see what the data looks like. 

In [ ]:
results_df

## Measurements

We calculate what proportion of the time the two methods gave the same result. 

In [ ]:
# Calculate proportion where they agreed. 

(results_df['weighted_winner'] == results_df['uniform_winner']).mean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi

# Define data
categories = ['Giuseppe Destefanie', 'Vasily Sumanov', 'Roderick McKinley']
values = [50, 30, 40]

# Number of variables
N = len(categories)

# Compute angle of each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Append the first value to the end to close the loop
values += values[:1]

# Initialize radar chart
ax = plt.subplot(111, polar=True)

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], categories)

# Draw y-labels
ax.set_rlabel_position(0)
plt.yticks([10, 20, 30, 40, 50], ["10%", "20%", "30%", "40%", "50%"], color="grey", size=7)
plt.ylim(0, 50)

# Plot data
ax.plot(angles, values, linewidth=2, linestyle='solid')

# Fill area
ax.fill(angles, values, 'b', alpha=0.1)

# Show the chart
plt.show()


In [ ]:

# Number of variables
N = len(categories)

# Compute angle of each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Append the first value to the end to close the loop
values += values[:1]

# Initialize radar chart
ax = plt.subplot(111, polar=True)

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], categories)

# Draw y-labels
ax.set_rlabel_position(0)
plt.yticks([10, 20, 30, 40, 50], ["10%", "20%", "30%", "40%", "50%"], color="grey", size=7)
plt.ylim(0, 50)

# Plot data
ax.plot(angles, values, linewidth=2, linestyle='solid')

# Fill area
ax.fill(angles, values, 'b', alpha=0.1)

# Show the chart
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define data




df = pd.DataFrame(
    dict(
        categories = ['Giuseppe Destefanie', 'Vasily Sumanov', 'Roderick McKinley', 'Candidate Four'],
        values = [50, 30, 40, 20])
        )
fig = px.line_polar(df, r='values', theta='categories', line_close=True)
fig.show()

In [ ]:

Here is an example code snippet in Python using [pandas](file:///c%3A/Users/andre/Desktop/basic-voting-calc/notebooks/experiment_template.ipynb#37%2C8-37%2C8) to read a .csv file from a Google Sheet link:

In [ ]:
import pandas as pd

# Replace 'YOUR_GOOGLE_SHEET_LINK' with the link to your published Google Sheet .csv file
google_sheet_link = 'https://docs.google.com/spreadsheets/d/1WJAV5O4otoTSH-E6wqJ8_GX4FdFO18GYB-F3b7IkKjA/edit?usp=sharing'

# Read the .csv file from the Google Sheet link
df = pd.read_csv(google_sheet_link)

# Now you can work with the data in the DataFrame 'df'